# Chapter 09: -5- Putting everything together!
2020-11-17

Content:
1. UML diagram of the unitconver package
2. setup.py and CLI.py for the entry point (and argParse module)
3. UnitTable.py and the unit_tables directory (and lambda function)
4. Converter.py (and importlib module)
5. Putting all of them together!

## 幾個重要的概念：
1. Chapter 05 用 **os.argv[**]**
2. Chapter 09 用 **argparse** 的方法：提供 positional/optional argument，之後作相對應的處理，其中並包括提示的 -h 等相關參數的選項資訊
3. **import importlib**: 如何 **run-time** 載入一個 py 檔的 module
4. **getattr()** 如何 **run-time instatntiate** 一個指定類別的 object？從一個 module 中，取出其中定義的一個 class 的 type，然後利用在 **type 後馬上補上()** 就可 instantiate 一個相對應的物件
5. **lambda function**: 同一名稱的 function 可透過 (key) 調整適應不同的狀況，舉例來說，像是 to_base_unit 的 source。我們可以將這些 lambda function 按 key 存入一個 dictionary，之後用 key 取得。在此之前，用前述取得物件的技術，先取得藏這 dictionary 的物件。
4. **inspect module** 在我們執要環境中的 modules, classes, methods, functions,...等，一定都是從 code 來的，我們會想要知道其相對應的 code 為何？

In [1]:
import os
course_directory = 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release'
os.chdir(course_directory + "/Chapter09/unitconverter")

## -5- 現在全部放在一起，再看一遍

In [2]:
# import required libraries
import argparse
import inspect
from unitconverter.Converter import get_table, convert_units

### 2.1 借助 python 內建的 setup 功能，可以方便地構建CLI環境。
setup.py定義了以下內容：
+ entry_points: 我們可以在CLI界面中輸入"unitconvert" 的指令來啟動這個程式
+ packages: 需要包含進來的所有程式

### 注意：setup.py 的程式無法在 jupyter 環境中執行

In [ ]:
'''
以下是 setup.py 的內容

在 ipython 裏執行會出錯，必須在 shell 中執行
'''
from setuptools import setup

setup(
    name='unitconverter',
    version='0.1.0',
    entry_points={
        'console_scripts': ['unitconvert=unitconverter.CLI:run_cli'],
    },  ## 這一行 entry_points 是新加的！ 以後就在 command line 打 unitconvert 即可
    description='Command line tool for unit conversion',
    classifiers=[
        'Natural Language :: English',
        'Programming Language :: Python :: 3.5',
        'Programming Language :: Python :: 3.6',
        'Programming Language :: Python :: 3.7',
    ],
    author='Dan Nixon',
    packages=['unitconverter', 'unitconverter.unit_tables'],
    include_package_data=True, #  to include any data files in package
    zip_safe=False)


我們可以使用 shell command 的提非號 ! \
而不需轉到 console，即可執行 "python setup.py install"，\
安裝完以後方可以啟用命令行工具

In [3]:
!tree

列出磁碟區 新增磁碟區 的資料夾 PATH
磁碟區序號為 E422-9B08
D:.
├─build
│  ├─bdist.win-amd64
│  └─lib
│      └─unitconverter
│          └─unit_tables
├─dist
├─unitconverter
│  ├─unit_tables
│  │  └─__pycache__
│  └─__pycache__
├─unitconverter.egg-info
└─__pycache__ (1)


In [4]:
! python setup.py install

running install
running bdist_egg
running egg_info
writing unitconverter.egg-info\PKG-INFO
writing dependency_links to unitconverter.egg-info\dependency_links.txt
writing entry points to unitconverter.egg-info\entry_points.txt
writing top-level names to unitconverter.egg-info\top_level.txt
reading manifest file 'unitconverter.egg-info\SOURCES.txt'
writing manifest file 'unitconverter.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build\bdist.win-amd64\egg
copying build\lib\desktop.ini -> build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\unitconverter
copying build\lib\unitconverter\CLI.py -> build\bdist.win-amd64\egg\unitconverter
copying build\lib\unitconverter\Converter.py -> build\bdist.win-amd64\egg\unitconverter
copying build\lib\unitconverter\desktop.ini -> build\bdist.win-amd64\egg\unitconverter
copying build\lib\unitconverter\UnitTable.py -> build\bdist.win-amd64\egg\unitconverter
creating buil

In [7]:
! unitconvert -h

usage: unitconvert [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [5]:
! unitconvert time list -m

Unit table time can convert between the units:
d (x / 86400.0)
y (x / 3.15569e7)
ns (x * 1000000000.0)
s (base unit)
h (x / 3600.0)
m (x / 60.0)
ms (x * 1000.0)
us (x * 1000000.0)


In [6]:
! unitconvert time convert -h

usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]

positional arguments:
  VALUE       The value to convert
  FROM        Unit to convert from
  TO          Unit(s) to convert to

optional arguments:
  -h, --help  show this help message and exit


## 以下是本章的重頭戲，仔細說明 CLI.py 的內容

**整個的重點就是怎麼 (解譯) command line 中讀進來的指令以及接在後面的各個 positional/optional arguments → propos**

- parser.add_argument()
- subparsers = parser.add_subparsers()
- list_table_parser = subparsers.add_parser('list')
- list_table_parser.set_defaults(which='list')
- list_table_parser.add_argument()

In [ ]:
'''
以下內容是 CLI.py 內容
'''
import argparse
import inspect
from unitconverter.Converter import get_table, convert_units


def run_cli(): # 這是在 setup.py 裏面加進 entry_ponts 的執行程式主體
    parser = argparse.ArgumentParser(description='Tool for converting units')

    parser.add_argument( # 第一個參數
        'table',         # 存入物件的 attribute
        metavar='TABLE',
        action='store',
        type=str,        # 資料型態
        help='Unit table to use in conversion'
    )

    ## 接下來的參數可能是 {list, convert} 它們通稱 subparsers
    
    subparsers = parser.add_subparsers(help='operation to be performed')

    list_table_parser = subparsers.add_parser('list')
    list_table_parser.set_defaults(which='list')

    list_table_parser.add_argument(
        '-m', '--method',     # 多一個 help 的選項：如果有的話，設定為 TRUE，之後會用到
        action='store_true',
        help='Also output the conversion method from the base unit'
    )

    conversion_parser = subparsers.add_parser('convert')
    conversion_parser.set_defaults(which='convert')

    ## 之後，如果是要 conver 的話，後面還要有 VALUE FROM TO (+TO)
    
    ## 這些為 convert 修件下的 positional argument
    ## positional 表示與出現的順序有關
    
    conversion_parser.add_argument(
        'value',
        metavar='VALUE',
        type=float,
        action='store',
        help='The value to convert'
    )

    conversion_parser.add_argument(
        'from_unit',
        metavar='FROM',
        action='store',
        type=str,
        help='Unit to convert from'
    )

    conversion_parser.add_argument(
        'to_units',
        metavar='TO',
        action='store',
        nargs='+', # 至少有一個
        type=str,
        help='Unit(s) to convert to'
    )

    ## 所有輸入的參數入列到 propos 裏
    ## 在上面一共有看到 {which, method, value, from_unit, to_units }
    props = parser.parse_args()

    if props.which == 'list':
        _run_unit_list(props)
    elif props.which == 'convert':
        _run_conversion(props)


def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print('Unit table %s can convert between the units:' % props.table)
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                '''
                還記得 lambda function 嗎？
                        self.to_base_unit['ev']        = lambda x: x / 6241506480000000000.0
                        self.from_base_unit['ev']      = lambda x: x * 6241506480000000000.0
                '''                           
                conversion = inspect.getsource(table.from_base_unit[unit])
                formula = conversion[conversion.index(':')+1:conversion.index('\n')].strip()
            print('%s (%s)' % (unit, formula))
        else:
            print(unit)


def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    for result in results:
        print('%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit']))


### 我們先看：對 "unitconvert table list -- method" 的處理方式

In [8]:
!unitconvert energy list -m

Unit table energy can convert between the units:
ev (x * 6241506480000000000.0)
btu (x * 0.00094781707775)
kcal (x * 0.00023884589663)
wh (x * 0.00027777777778)
hph (x * 3.7250614123e-7)
cal (x * 0.23900573614)
j (base unit)


In [9]:
get_table("time").__dict__

{'base_unit': 's',
 'to_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>},
 'from_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': 

In [10]:
get_table("time").from_base_unit

{'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>}

In [11]:
'''
time 這個物件的 from_base_unit['d'] 的這個變數，是一個 lambda function
'''
get_table("time").from_base_unit['d']

<function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>

In [12]:
'''
inspect 是用來將其原始的程式碼取出來
'''
import inspect
conversion = inspect.getsource(get_table("time").from_base_unit['d'])
conversion

"        self.from_base_unit['d'] = lambda x: x / 86400.0\n"

In [13]:
'''
以下是當選用 list 時的反應：
'''
def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print('Unit table %s can convert between the units:' % props.table)
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                conversion = inspect.getsource(table.from_base_unit[unit])
                """
                self.from_base_unit['radian'] = lambda x: x * 0.0174532925
                """
                formula = conversion[conversion.index(':') + 1:conversion.index('\n')].strip() # 這是要將一開頭的 [lambda x:] 給去掉
            print('%s (%s)' % (unit, formula))
        else:
            print(unit)


In [14]:
'''
conversion 是一個字串，我們取出 "lambda x: x / 86400.0\n" 中的 "x / 86400.0"
'''
formula = conversion[conversion.index(':')+1:conversion.index('\n')].strip()
formula

'x / 86400.0'

In [15]:
! unitconvert time list -m

Unit table time can convert between the units:
d (x / 86400.0)
y (x / 3.15569e7)
s (base unit)
ms (x * 1000.0)
m (x / 60.0)
us (x * 1000000.0)
h (x / 3600.0)
ns (x * 1000000000.0)


In [16]:
convert_units('time', 1, 'd', ["m", "s"])

[{'dest_unit': 'm', 'converted_value': 1440.0},
 {'dest_unit': 's', 'converted_value': 86400.0}]

## 我們再看其他的幾個例子

In [17]:
!unitconvert energy list

Unit table energy can convert between the units:
j
ev
kcal
hph
btu
cal
wh


In [18]:
!unitconvert energy list -m

Unit table energy can convert between the units:
ev (x * 6241506480000000000.0)
btu (x * 0.00094781707775)
j (base unit)
cal (x * 0.23900573614)
hph (x * 3.7250614123e-7)
wh (x * 0.00027777777778)
kcal (x * 0.00023884589663)


In [19]:
! unitconvert energy convert 2500 kcal cal j ev

2500.000000 kcal = 2501673.040151 cal
2500.000000 kcal = 10466999.999890 j
2500.000000 kcal = 65329848325475080695971840.000000 ev


## 2.2 主幹 parser 

1. 之前在 Chapter 05 中，有寫過 CLI.py 是用 argv[-] 來處理輸入的參數，\
還記得嗎？  for arg in sys.argv[1:]:

2. 現在，用另外一種方式，先宣告一個 parser 用於儲存 table 的類別，\
之後所有的輸入都會進這個物件裏，而且，我們也可以準備一些類似 --help 等的資訊

In [20]:
'''
這是 CLI.py 部份內容
'''
import argparse
from unitconverter.Converter import get_table, convert_units
import inspect ## 之後我們要將 lambda function 的指令讀出來用

parser = argparse.ArgumentParser(description='Tool for converting units')

parser.add_argument(
    'table',
    metavar='TABLE',
    action='store',
    type=str,
    help='Unit table to use in conversion'
)

_StoreAction(option_strings=[], dest='table', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Unit table to use in conversion', metavar='TABLE')

### 另外定義 list 和 convert 兩種功能，依靠關鍵字 "list" 和 "convert" 以在兩者之間切換

In [21]:
subparsers = parser.add_subparsers(help='operation to be performed')

list_table_parser = subparsers.add_parser('list')
list_table_parser.set_defaults(which='list')

In [22]:
conversion_parser = subparsers.add_parser('convert')
conversion_parser.set_defaults(which='convert')

## 變數props儲存用戶輸入的功能關鍵字，利用它來進行功能切換

In [23]:
!unitconvert -h

usage: unitconvert [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [24]:
# 以下是 CLI.py 針對不同的 which 進行不相應的處理流程：
# 在 jupyter notebook 中，不能單獨執行
props = parser.parse_args()

if props.which == 'list':
    _run_unit_list(props)
elif props.which == 'convert':
    _run_conversion(props)

usage: ipykernel_launcher.py [-h] TABLE {list,convert} ...
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\Public\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 2.3 list功能
#### 對於list功能，新增一個argument -m，讓程式可以輸出不同單位之間的轉換關係

In [25]:
!unitconvert time list -h

usage: unitconvert TABLE list [-h] [-m]

optional arguments:
  -h, --help    show this help message and exit
  -m, --method  Also output the conversion method from the base unit


In [26]:
!unitconvert time list -m

Unit table time can convert between the units:
y (x / 3.15569e7)
d (x / 86400.0)
s (base unit)
h (x / 3600.0)
ns (x * 1000000000.0)
m (x / 60.0)
ms (x * 1000.0)
us (x * 1000000.0)


In [27]:
list_table_parser.add_argument(
        '-m', '--method',
        action='store_true',
        help='Also output the conversion method from the base unit'
    )

_StoreTrueAction(option_strings=['-m', '--method'], dest='method', nargs=0, const=True, default=False, type=None, choices=None, help='Also output the conversion method from the base unit', metavar=None)

##### 查看 energy table 中所有的單位

In [29]:
!unitconvert energy list

Unit table energy can convert between the units:
kcal
hph
cal
wh
j
btu
ev


##### 查看 energy table 中單位之間的轉換關係

In [30]:
!unitconvert energy list -m

Unit table energy can convert between the units:
j (base unit)
wh (x * 0.00027777777778)
hph (x * 3.7250614123e-7)
btu (x * 0.00094781707775)
ev (x * 6241506480000000000.0)
kcal (x * 0.00023884589663)
cal (x * 0.23900573614)


## list功能本質上是依靠 \_run\_unit\_list方法來實現的
    *呼叫Converter.py中的get_table方法
    *當沒有argument -m傳入時，props.method為False，此時僅輸出各單位名稱
    *當有argument -m傳入時，props.method為True，此時輸出各個單位間的轉換關係

In [31]:
def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print('Unit table %s can convert between the units:')
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                conversion = inspect.getsource(table.from_base_unit[unit])
                formula = conversion[conversion.index(':') + 1:conversion.index('\n')].strip()
            print('%s (%s)' % (unit, formula))
        else:
            print(nit)

### 仔細看看\_run\_unit\_list方法的實現細節

In [32]:
table = get_table('energy')
unit = 'ev'
table.from_base_unit[unit]

<function unitconverter.unit_tables.energy.energy.__init__.<locals>.<lambda>(x)>

In [33]:
conversion = inspect.getsource(table.from_base_unit[unit])
conversion

"        self.from_base_unit['ev'] = lambda x: x * 6241506480000000000.0\n"

In [34]:
conversion[conversion.index(':') + 1:conversion.index('\n')]

' x * 6241506480000000000.0'

In [35]:
conversion[conversion.index(':') + 1:conversion.index('\n')].strip()

'x * 6241506480000000000.0'

## 2.4 convert 功能
### 對於 convert 功能，新增 positional arguments，用來儲存單位轉換中的相關數值

In [36]:
conversion_parser.add_argument(
    'value',
    metavar='VALUE',
    type=float,
    action='store',
    help='The value to convert'
)

conversion_parser.add_argument(
    'from_unit',
    metavar='FROM',
    action='store',
    type=str,
    help='Unit to convert from'
)

conversion_parser.add_argument(
    'to_units',
    metavar='TO',
    action='store',
    nargs='+',
    type=str,
    help='Unit(s) to convert to'
)

_StoreAction(option_strings=[], dest='to_units', nargs='+', const=None, default=None, type=<class 'str'>, choices=None, help='Unit(s) to convert to', metavar='TO')

### 查看幫助

In [37]:
!unitconvert energy convert -h

usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]

positional arguments:
  VALUE       The value to convert
  FROM        Unit to convert from
  TO          Unit(s) to convert to

optional arguments:
  -h, --help  show this help message and exit


##### 類似 convert_units('energy', 200, 'wh', ['j', 'hph']) 的用法，使用 CLI 進行單位轉換
    注意順序

In [38]:
! unitconvert energy convert 2500 kcal cal j ev

2500.000000 kcal = 2501673.040151 cal
2500.000000 kcal = 10466999.999890 j
2500.000000 kcal = 65329848325475080695971840.000000 ev


#### convert 功能本質上是依靠 \_run_conversion 方法來實現的
    *呼叫Converter.py中的convert_units方法

In [39]:
def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    for result in results:
        print('%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit']))

In [40]:
convert_units("energy", 2500, "kcal", ["cal", "j", "ev"])

[{'dest_unit': 'cal', 'converted_value': 2501673.0401511528},
 {'dest_unit': 'j', 'converted_value': 10466999.999890264},
 {'dest_unit': 'ev', 'converted_value': 6.532984832547508e+25}]